In [1]:
from cosecha_colectiva import miscelaneous as m
m.try_find_conf_file()

from cosecha_colectiva import main_tester as mt
from cosecha_colectiva import query_cosecha as qc
from cosecha_colectiva import config
from cosecha_colectiva import CAF_DB_Monitor as CDM
from cosecha_colectiva import test_cosecha_api as tca

import logging
import pathlib
from datetime import datetime
import pandas as pd

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


[2023-04-19 12:28:19,058][INFO]: Connecting b3cf7edd2f8490@us-cdbr-east-06.cleardb.net:3306
[2023-04-19 12:28:19,839][INFO]: Connected b3cf7edd2f8490@us-cdbr-east-06.cleardb.net:3306


In [2]:
#xls_name = 'CAF_GrupoTesteador.xlsx'
xls_name = 'CAF_GrupoAstronomo.xlsx'
#xls_name = 'CAF_GrupoDomador.xlsx'
xls_name = 'CAF_GrupoMesero.xlsx'

In [3]:
id_grupo = qc.get_grupo_id_by_name(xls_name.split('.')[0])
print(id_grupo)

1294


In [4]:
qc.delete_grupo(id_grupo, solo_sesiones=True, force_delete=True)

([{'Transaccion_prestamo_id': 3434}, {'Transaccion_prestamo_id': 3444}, {'Transaccion_prestamo_id': 3454}, {'Transaccion_prestamo_id': 3464}, {'Transaccion_prestamo_id': 3474}, {'Transaccion_prestamo_id': 3484}, {'Transaccion_prestamo_id': 3494}, {'Transaccion_prestamo_id': 3504}, {'Transaccion_prestamo_id': 3514}], 'transaccion_prestamos')
DELETE from heroku_59f792929604e63.transaccion_prestamos WHERE Transaccion_prestamo_id in (3434, 3444, 3454, 3464, 3474, 3484, 3494, 3504, 3514)
([{'Transaccion_id': 16854}, {'Transaccion_id': 16864}, {'Transaccion_id': 16874}, {'Transaccion_id': 16884}, {'Transaccion_id': 16894}, {'Transaccion_id': 16904}, {'Transaccion_id': 16914}, {'Transaccion_id': 16924}, {'Transaccion_id': 16934}, {'Transaccion_id': 16944}, {'Transaccion_id': 16954}], 'transacciones')
DELETE from heroku_59f792929604e63.transacciones WHERE Transaccion_id in (16854, 16864, 16874, 16884, 16894, 16904, 16914, 16924, 16934, 16944, 16954)
([{'Interes_prestamo_id': 1904}, {'Interes_p

In [5]:
now = datetime.now()
log_date = now.strftime("%Y-%m-%d_%H_%M_%S")
log_file = pathlib.Path(config.log_dir, log_date+'_Sesiones.log')
logging.basicConfig(filename=log_file, encoding='utf-8', level=logging.DEBUG)

In [6]:
monitor_grupo = CDM.CAF_DB_Monitor(id_grupo)

In [8]:
start_month = 7
end_month = 9

for month in range(start_month, end_month+1):

    print('month: ', month)

    try:
        monitor_grupo.actualiza_todo_sesion(xls_name, month, type_xls='NEW')
    except Exception as e:
        print(e)
        raise(e)
    try:
        mt.test_sesion_months(xls_name, month, month, type_xls='NEW')
    except Exception as e:
        print(e)
    [status, dfs] = monitor_grupo.compara_db_monitor()
    #if not status:
    #    print('Algo no coincidió')
    #    break

month:  7
prestamos No coinciden monitor y db
month:  8
prestamos No coinciden monitor y db
month:  9
prestamos No coinciden monitor y db


In [9]:
monitor_grupo.interes_prestamo

,Interes_prestamo_id,Prestamo_id,Sesion_id,Monto_interes,Tipo_interes
0,1994,2314,5314,10.0,0
1,2004,2314,5324,20.0,0
2,2014,2324,5324,10.0,0
3,2024,2314,5334,30.0,0
4,2034,2324,5334,20.0,0
5,2044,2314,5344,40.0,0
6,2054,2324,5344,30.0,0
7,2064,2314,5354,57.5,1
8,2074,2324,5354,40.0,0
9,2084,2314,5364,75.0,1


In [10]:
monitor_grupo.prestamos

,Prestamo_id,Monto_prestamo,Monto_pagado,Interes_generado,Interes_pagado,Estatus_ampliacion,Num_sesiones,Sesiones_restantes,Estatus_prestamo,Socio_id,...,Interes_morosidad,Ampliacion_prestamos,Interes_ampliacion,Mod_calculo_interes,Tasa_interes_prestamo_grande,interes_futuro,Ultimo_interes_pagado,debe_interes,sobrante_abono,Ultimo_abono
0,2314,500.0,290.0,110.0,110.0,False,4,-4,0,1234,...,1.5,1,1.5,1,1.5,17.5,17.5,17.5,32.5,50.0
1,2324,500.0,500.0,50.0,50.0,False,6,1,1,1244,...,1.5,1,1.5,1,1.5,0.0,0,0,0,0


In [13]:
monitor_grupo.ganancias.tail(30)

,Ganancias_id,Monto_ganancia,Entregada,Socio_id,Sesion_id
60,37264,2.75,0,1194,5364
61,37274,2.75,0,1204,5364
62,37284,2.75,0,1214,5364
63,37294,2.75,0,1224,5364
64,37304,2.75,0,1234,5364
65,37314,2.75,0,1244,5364
66,37324,2.75,0,1254,5364
67,37334,2.75,0,1264,5364
68,37344,2.75,0,1274,5364
69,37354,2.75,0,1284,5364
